In [20]:

#Packages
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta

try:
    # Set end date to today and start date to 6 months ago
    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)
    
    print(f"Downloading data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

    # Download data for each stock separately with progress tracking
    stocks = ['0050.TW', '0056.TW', '00662.TW', '00701.TW', '00702.TW',
              '00703.TW', '00757.TW', '00830.TW', '00850.TW', '00878.TW',
              '00881.TW', '00882.TW', '00891.TW', '00893.TW', '00894.TW',
              '00915.TW', '00918.TW', '00919.TW', '00927.TW', '00929.TW',
              '00934.TW', '00936.TW', '00939.TW', '00940.TW', '00944.TW',
              '00946.TW', '00954.TW', '2330.TW', '^TWII']

    # Download data with verification
    data_dict = {}
    for stock in stocks:
        print(f"Downloading {stock}...", end=' ')
        df = yf.download(stock, start=start_date, end=end_date, progress=False)
        if len(df) > 0:
            data_dict[stock] = df['Close']
            print(f"Success ({len(df)} rows)")
        else:
            print("Failed")

    # Combine all successful downloads
    df_combined = pd.concat([data_dict[stock] for stock in data_dict], axis=1)
    df_combined.columns = list(data_dict.keys())

    # Calculate daily returns
    returns = df_combined.pct_change().dropna()

    # Calculate beta for each stock
    betas = {}
    r_squared = {}
    for column in returns.columns[:-1]:  # Exclude TWII
        if column != '^TWII':
            slope, intercept, r_value, p_value, std_err = stats.linregress(returns['^TWII'], returns[column])
            betas[column] = slope
            r_squared[column] = r_value**2

    # Create DataFrame with results
    beta_results = pd.DataFrame({
        'Beta': betas,
        'R-squared': r_squared
    })

    # Sort by absolute beta value
    beta_results['Abs_Beta'] = abs(beta_results['Beta'])
    beta_results = beta_results.sort_values('Abs_Beta', ascending=False)
    beta_results = beta_results.drop('Abs_Beta', axis=1)

    # Print results with better formatting
    print("\nBeta Analysis Results:")
    print("=====================")
    pd.set_option('display.float_format', lambda x: '%.4f' % x)  # Set to 4 decimal places
    print(beta_results)

    # Print some summary statistics
    print("\nSummary Statistics:")
    print("==================")
    print(f"Average Beta: {beta_results['Beta'].mean():.4f}")
    print(f"Median Beta: {beta_results['Beta'].median():.4f}")
    print(f"Average R-squared: {beta_results['R-squared'].mean():.4f}")
    
    # Print the number of stocks analyzed
    print(f"\nTotal stocks analyzed: {len(beta_results)}")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    import traceback
    print("\nDetailed error information:")
    print(traceback.format_exc())



Beta Analysis Results:
           Beta  R-squared
2330.TW  1.4680     0.8585
00830.TW 1.1725     0.5069
00894.TW 1.1029     0.8361
0050.TW  1.0740     0.9074
00881.TW 1.0212     0.8684
00891.TW 0.9770     0.7290
00850.TW 0.8757     0.9191
00954.TW 0.8659     0.2754
00893.TW 0.8388     0.3437
00927.TW 0.8062     0.7405
00757.TW 0.7379     0.3523
00662.TW 0.6371     0.4178
0056.TW  0.5613     0.7321
00878.TW 0.5160     0.7701
00934.TW 0.5094     0.5725
00929.TW 0.4720     0.5530
00918.TW 0.4672     0.6100
00940.TW 0.4639     0.6338
00946.TW 0.4582     0.5236
00919.TW 0.4540     0.5367
00939.TW 0.4247     0.6205
00944.TW 0.4173     0.6513
00936.TW 0.3814     0.4986
00915.TW 0.3611     0.5202
00701.TW 0.3266     0.4385
00703.TW 0.1970     0.0080
00882.TW 0.1705     0.0089
00702.TW 0.0968     0.0349

Summary Statistics:
Average Beta: 0.6377
Median Beta: 0.5127
Average R-squared: 0.5524

Total stocks analyzed: 28


C:\Users\user\AppData\Local\Temp\ipykernel_16676\1611608007.py:39: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df_combined.pct_change().dropna()
